<font size="6">PyCoinGecko for CoinGecko API</font>

Cryptocurrency data is excellent to be used in an API because it is constantly updated and it is vital to cryptocurrency trading. We will use the **PyCoinGecko Python client/wrapper** for the **CoinGecko API**, updated every minute by CoinGecko. We use the wrapper/client because it is easy to use so you can focus on the task of collecting data.

Using **PyCoinGecko** to collect data is simple. All we need is to install and import the library, then create a client object, and finally use a function to request our data.

First, we need to install the **PyCoinGecko** library. Uncomment the following code block if the **PyCoinGecko** library has not been installed in the machine.

In [2]:
# !pip install pycoingecko

Then we need to import the library and create a client object.

In [3]:
# Import the library
from pycoingecko import CoinGeckoAPI

In [4]:
# Create the client object
cg = CoinGeckoAPI()

Finally, we use a function to request our data. In this function we are getting data on **bitcoin**, in **US dollars**, for the past **30** days. 

In [5]:
# Use the function to request the data
bitcoin_data = cg.get_coin_market_chart_by_id(id = 'bitcoin', vs_currency = 'usd', days = 30)

In this case our response is a **JSON** expressed as a **Python dictionary** of nested lists including _price_, _market cap_, and _total volumes_, which contain the _UNIX timestamp_ and the price at that time.

For easy viewing, we convert the Python dictionary into a Pandas dataframe.

In [6]:
# Import pandas library
import pandas as pd

In [7]:
df_bitcoin_data = pd.DataFrame(bitcoin_data)

In [8]:
df_bitcoin_data.head()

,prices,market_caps,total_volumes
0,"[1713154190681, 65183.74548356216]","[1713154190681, 1282757178488.9004]","[1713154190681, 45616702202.22862]"
1,"[1713157404248, 64977.77063893974]","[1713157404248, 1279055378783.3743]","[1713157404248, 45076719918.31025]"
2,"[1713161090159, 65535.173579147835]","[1713161090159, 1286403947858.7358]","[1713161090159, 38294871961.13445]"
3,"[1713164616618, 66570.16139354806]","[1713164616618, 1307780123641.0579]","[1713164616618, 45750163114.96653]"
4,"[1713168269240, 66361.17474265248]","[1713168269240, 1306397006932.165]","[1713168269240, 45542653103.29848]"


We are only interested in price so that is what we will select using the key `price`. To make things simple, we can convert our nested list to a `DataFrame`, with the columns `timestamp` and `price`.

The following codes select the `prices` column from the Python dictionary and make the column heading `TimeStamp` and `Price`.

In [9]:
# Extracting the prices column from the Python dictionary
data = pd.DataFrame(bitcoin_data['prices'], columns=['TimeStamp', 'Price'])
data.head()

,TimeStamp,Price
0,1713154190681,65183.745484
1,1713157404248,64977.770639
2,1713161090159,65535.173579
3,1713164616618,66570.161394
4,1713168269240,66361.174743


It's difficult to understand the column `timestamp`. We will convert it to a more readable format using the pandas function `to_datetime()`. Using the `to_datetime()` function, we create readable time data, the input is the `timestamp` column, unit of time is set to milliseconds. We append the output to the new column, `date`. 

In [10]:
# Convert the timestamp to a readable format
data['Date'] = pd.to_datetime(data['TimeStamp'], unit = 'ms')
data.head()

,TimeStamp,Price,Date
0,1713154190681,65183.745484,2024-04-15 04:09:50.681
1,1713157404248,64977.770639,2024-04-15 05:03:24.248
2,1713161090159,65535.173579,2024-04-15 06:04:50.159
3,1713164616618,66570.161394,2024-04-15 07:03:36.618
4,1713168269240,66361.174743,2024-04-15 08:04:29.240


Now we want to create a candlestick plot. To get the data for the daily candlesticks we will group by the date to find the minimum, maximum, first, and last price of each day. (Please refer to the end of this notebook for the usage of Pandas Series `dt.date()` function.

In [11]:
# Get the candlestick plot data
candlestick_data = data.groupby(data.Date.dt.date).agg({'Price': ['min', 'max', 'first', 'last']})
candlestick_data.head()

Price                                          
                     min           max         first          last
Date                                                              
2024-04-15  63190.368481  66650.479411  65183.745484  63415.485289
2024-04-16  61922.025495  63937.600851  63409.243114  63812.814757
2024-04-17  60181.669230  64238.956490  63813.800580  61657.579437
2024-04-18  61048.751769  63702.626304  61271.934846  63524.558683
2024-04-19  61055.956877  65079.822640  63517.289856  64112.513451

Finally we will use `plotly` to create the candlestick chart and plot it.

We need to install the `plotly` library before we can use it. Uncomment the following code block if `plotly` has not been installed in the machine. You can obtain more information from [Getting Started with Plotly in Python](https://plotly.com/python/getting-started/).

In [12]:
# !pip install plotly

In [13]:
# Import the library to create the plot
import plotly.graph_objects as go

In [14]:
# Import the library to create a html file of the plot
import plotly.offline

In [15]:
plotlyfig = go.Figure(data=[go.Candlestick(x = candlestick_data.index,
                                           open = candlestick_data['Price']['first'],
                                           high = candlestick_data['Price']['max'],
                                           low = candlestick_data['Price']['min'],
                                           close = candlestick_data['Price']['last']
                                           )
                            ]
                     )

plotlyfig.update_layout(xaxis_rangeslider_visible = False, xaxis_title = 'Date',
                        yaxis_title = 'Price (USD $)', title = 'Bitcoin Candlestick Chart Over Past 30 Days')

plotly.offline.plot(plotlyfig, filename = './bitcoin_candlestick_graph.html', auto_open=False)

'./bitcoin_candlestick_graph.html'

The [HTML file of the chart](https://nbviewer.org/github/stevenkhwun/IBM_Data-Science/blob/main/My_Notebook/bitcoin_candlestick_graph.html) was created. Commenting out the last code in the last code block, we can view the candlestick chart in the Jupyter Notebook cell.

<font size="4">The `Plotly` library</font>

You can find more information in using the `Plotly` library from the following website:
* [Getting Started with Plotly in Python](https://plotly.com/python/getting-started/)
* [Plotly Open Source Graphing Library for Python](https://plotly.com/python/)
* [Displaying Figures in Python](https://plotly.com/python/renderers/)
* [Troubleshooting Plotly Chart Not Showing in Jupyter Notebook and Jupyter Lab](https://saturncloud.io/blog/troubleshooting-plotly-chart-not-showing-in-jupyter-notebook/)

<font size="4">Pandas Series `dt.date` attribute</font>

The `dt.date` attribute extracts the date part of the `DateTime` objects in a Pandas Series.

It returns the NumPy array of Python `datetime.date` objects, mainly the date part of timestamps without information about the time and timezone. Please refer to this [GeeksforGeeks](https://www.geeksforgeeks.org/pandas-series-dt-date/) for more information.

In [16]:
# Example
import pandas as pd 
sr = pd.Series(['2012-10-21 09:30', '2019-7-18 12:30', '2008-02-2 10:30', 
			'2010-4-22 09:25', '2019-11-8 02:22']) 
idx = ['Day 1', 'Day 2', 'Day 3', 'Day 4', 'Day 5'] 
sr.index = idx 
sr = pd.to_datetime(sr) 
result = sr.dt.date 
print(result)

Day 1    2012-10-21
Day 2    2019-07-18
Day 3    2008-02-02
Day 4    2010-04-22
Day 5    2019-11-08
dtype: object
